# Disseration Experiment 
# XAI Metrics - Function Implementations
Ciaran Finnegan October 2023

## IDENTITY

### Pseudocode

- Start with first instance in test data
- Search all other instances in test data and calculate distance from first instance (feature distance)
- Select closest other instance to first instance, i
- Generate explanations for all instances in test data
- Calculate distance of first instance explanations from explanations in all other instances
- Select closest other instance to first instance (explanation distance), t
- Generate success if instance id (i) = instance id (t)
- Drop first instance from test data

### Implementation

In [1]:
from scipy.spatial import distance

In [2]:
def get_identity_metric(features_df, xai_values_df, XAI_Type):
    """
    For each instance in the feature dataframe, this function identifies the closest instance 
    based on Euclidean distance. It then does the same for the corresponding XAI Explainor values. 
    The function checks if the closest instances for both features and XAI Explainor values match.
    
    Returns:
        Percentage of instances where the closest feature and XAI Explainor value instances match.
    """

    # Initialize match count to zero
    match_count = 0
    
    # Loop through each instance in the feature dataframe
    for idx, instance in features_df.iterrows():
        # Compute the Euclidean distance between the current instance and all other instances
        feature_distances = features_df.drop(index=idx).apply(lambda row: distance.euclidean(row, instance), axis=1)
        
        # Identify the index of the closest instance
        closest_feature_idx = feature_distances.idxmin()
        
        # Repeat the process for XAI Explanations
        xai_instance = xai_values_df.loc[idx]
        xai_distances = xai_values_df.drop(index=idx).apply(lambda row: distance.euclidean(row, xai_instance), axis=1)
        closest_xai_idx = xai_distances.idxmin()
        
        # Check if the closest instances for both features and XAI Explanations match
        if closest_feature_idx == closest_xai_idx:
            match_count += 1
        
        # Print the distances for debugging purposes
        print(f"Instance {idx}:   Current matches: {match_count}")
        print(f"\tClosest feature instance: {closest_feature_idx} (Distance: {feature_distances[closest_feature_idx]:.4f})")
        print(f"\tClosest "+ XAI_Type + " instance: {closest_xai_idx} (Distance: {xai_distances[closest_xai_idx]:.4f})")

    # Compute the matching percentage
    percentage = (match_count / len(features_df)) * 100
    print(f"\n\nThis is the function in XAI_METRICS_FUNCTIONS -- IDENTITY for " + XAI_Type + "\n")
    print(f"\n\nPercentage of matches: {percentage:.2f}%   {match_count} Matches of {len(features_df)} Entries")
    
    return percentage

## STABILITY

### Pseudocode

"Stability" - this metric states that instances belonging to the same class must have comparable explanations

- Assume that the dataset has been balanced 50:50 for fraud/non-fraud.
- Cluster explanations of all instances in test data by k-means, include the 'predicted fraud' label.
- Number of clusters equals label values, in this case two (fraud/non-fraud)
- For each instance in test data
	- compare explanation cluster label to predicted class label
	- if match, then stability satisfied
	
	alternatively
	
	- compare explanation cluster label in largest cluster to predicted class label
	- Take ratio of majority predicted class label to minority class as the stability measure (the higher the value the closer the explanation clusters map to 
	predicted results).	
	
Question: how do we know which explanations cluster equates to 'fraud' and which cluster equates to 'non-fraud'? If dataset is a 50:50 label split and we use 
two clusters then we can just pick one cluster (use the largest).


The training data is balanced but the Test data is not. 
The majority class in the Test data will be non-Fraud, so assume that is 
always the largest cluster.

### Implementation

In [3]:
def get_stability_metric(xai_values_df):
    """
    This function performs the following steps:
    1. Clusters the XAI Explainor values into two clusters using the k-means algorithm.
    2. Assigns the actual target value from the test dataset to each instance in the XAI Explainor values dataframe.
    3. Calculates the percentage of rows where the target class '0' matches the cluster value '0'.
    4. Outputs the final dataframe with cluster assignments and actual target values to a CSV file.
    
    Returns:
        Percentage of instances where target class '0' matches cluster value '0'.
    """
    
    # Cluster the XAI Explainor values into two clusters
    kmeans = KMeans(n_clusters=2, random_state=42).fit(xai_values_df)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Create a new dataframe with an additional column indicating the cluster assignment
    clustered_df = xai_values_df.copy()
    clustered_df['Cluster'] = cluster_labels
    
    # Rename clusters so that the largest cluster is always labeled '0'
    if sum(cluster_labels) > len(cluster_labels) / 2:
        clustered_df['Cluster'] = clustered_df['Cluster'].map({0: '1', 1: '0'})
    
    # Print the number of instances assigned to each cluster
    cluster_0_count = clustered_df[clustered_df['Cluster'] == '0'].shape[0]
    cluster_1_count = clustered_df[clustered_df['Cluster'] == '1'].shape[0]
    print(f"Number of Instances in Cluster '0': {cluster_0_count}")
    print(f"Number of Instances in Cluster '1': {cluster_1_count}")
    
    # Assign the appropriate subset of y_test values to the dataframe based on the selected indices
    clustered_df['Actual'] = y_test.loc[clustered_df.index].values
    
    # Calculate the percentage of rows where the target class '0' matches the cluster value '0'
    matches_0 = clustered_df[(clustered_df['Cluster'] == '0') & (clustered_df['Actual'] == 0)].shape[0]
    total_class_0 = clustered_df[clustered_df['Actual'] == 0].shape[0]
    
    # Calculate the percentage of rows where the target class '1' matches the cluster value '1'
    matches_1 = clustered_df[(clustered_df['Cluster'] == '1') & (clustered_df['Actual'] == 1)].shape[0]
    total_class_1 = clustered_df[clustered_df['Actual'] == 1].shape[0]
    
    # Print the results for class '0'
    print(f"\nFor Class '0':")
    print(f"Total Instances: {total_class_0}")
    print(f"Matching Cluster '0' Instances: {matches_0}")
    
    # Print the results for class '1'
    print(f"\nFor Class '1':")
    print(f"Total Instances: {total_class_1}")
    print(f"Matching Cluster '1' Instances: {matches_1}")
    
    # Output the final dataframe to a CSV file
    clustered_df.to_csv('clustered_stability.csv', index=True)
    print("\nOutput saved to 'clustered_stability.csv'")
    
    # Compute the matching percentage
    percentage = (matches_0 / total_class_0) * 100
    iOverallTotal = total_class_1 + total_class_0
    print(f"\n\nThis is the function in XAI_METRICS_FUNCTIONS -- STABILITY\n")
    print(f"\nPercentage of matches: {percentage:.2f}%   {percentage} Matches of {iOverallTotal} Entries")
    
    return percentage